# Calculate biogeographic traits

Our focus is on the spatial distribution of microbial biogeographic traits, so we also need to calculate several microbial community measures. The [microgeo](https://github.com/ChaonanLi/microgeo) R package provides some basic functions for calculating microbial relative abundance, diversity, and community assembly mechanism indicators. Of course, you can also visualize more microbial traits onto maps, not limited to the traits calculated by the [microgeo](https://github.com/ChaonanLi/microgeo) R package.

Here we need three R packages for this section of [microgeo](https://github.com/ChaonanLi/microgeo) R package tutorial. Just run the following codes to import them into R environment.

In [1]:
suppressMessages(require("magrittr")) 
require("ggplot2")  %>% suppressMessages()
require("microgeo") %>% suppressMessages()

If the Chinese characters cannot be displayed correctly, please run the following codes to set locale to `UTF-8`:

In [2]:
prev_locale <- Sys.setlocale("LC_CTYPE", "C.UTF-8") 

We need a standard microgeo dataset for the presentations in the section of tutorial.

In [3]:
# Using the map downloaded from DataV.GeoAtlas
data(qtp)
map <- read_aliyun_map(adcode = c(540000, 630000, 510000)) %>% suppressMessages() 
dataset.dts.aliyun <- create_dataset(mat = qtp$asv, ant = qtp$tax, met = qtp$met, map = map,
                                     phy = qtp$tre, env = qtp$env, lon = "longitude", lat = "latitude") 
dataset.dts.aliyun %<>% rarefy_count_table() 
dataset.dts.aliyun %<>% tidy_dataset()
dataset.dts.aliyun %>% show_dataset()

ℹ [2024-06-08 22:17:49] INFO ==> all samples fall within the map area!

ℹ [2024-06-08 22:17:49] INFO ==> dataset has been created successfully!

ℹ [2024-06-08 22:17:49] INFO ==> use `object %>% show_dataset()` to check the summary of dataset.

ℹ [2024-06-08 22:17:53] INFO ==> the ASV/gene abundance table has been rarefied with a sub-sample depth of 5310



── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 1. Calculate the relative abundance

We implemented a function of `calc_rel_abund()`, which calculates the relative abundance of ASVs/genes at each of classification level. Just run the following codes.

In [4]:
# Calculate relative abundance
dataset.dts.aliyun %<>% calc_rel_abund() %>% suppressMessages()
head(dataset.dts.aliyun$abd$raw$Phylum[,1:5])

,p__,p__Acidobacteria,p__Actinobacteria,p__Armatimonadetes,p__Bacteroidetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,0.1318267,13.16384,49.39736,0.00000000,2.090395
s2,0.1318267,13.46516,51.63842,0.03766478,1.902072
s3,0.1129944,21.92090,42.97552,0.00000000,1.374765
s4,0.1506591,15.85687,47.96610,0.03766478,4.726930
s5,0.1129944,12.97552,55.91337,0.03766478,1.167608
s6,0.2636535,15.31073,46.64783,0.03766478,2.749529


In [5]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 2. Calculate the ecological markers

We also designed a function of `calc_markers()`, which identifies the potentially ecological markers base on the relative abundance of ASV/genes at a given classification level. If there is only one environmnetal variable for the identification, then a correlation method would be used. If the are two or more environmental variables, then a mantel test would be applied. **Please note that such a function only works when the relative abundance of ASVs/genes are available in microgeo dataset!**. 

In [6]:
# Identify ecological markers based on soil pH in <env>[Correlation]
dataset.dts.aliyun %<>% calc_markers(use.var = 'pH', annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') 

ℹ [2024-06-08 22:18:01] INFO ==> found 18 ecological markers at `Phylum` level..

✔ [2024-06-08 22:18:01] SAVE ==> results have been saved to: object$abd$mar



In [7]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Acidobacteria,p__Acidobacteria,-0.3898299,2.023896e-46
p__Verrucomicrobia,p__Verrucomicrobia,-0.3700755,1.150345e-41
p__Rokubacteria,p__Rokubacteria,-0.3443490,5.956207e-36
p__Actinobacteria,p__Actinobacteria,0.3219090,2.188202e-31
p__Chloroflexi,p__Chloroflexi,0.2990333,4.091258e-27
p__Gemmatimonadetes,p__Gemmatimonadetes,0.2937886,3.457193e-26


In [8]:
# Identify ecological markers based on soil pH and TOC in <env> [Mantel test]
dataset.dts.aliyun %<>% calc_markers(use.var = c('pH', 'TOC'), annotation.level = 'Phylum', r.thres = 0.1, use.dat = 'env') 

ℹ [2024-06-08 22:18:01] INFO ==> mantel test would take a while...

ℹ [2024-06-08 22:18:01] INFO ==> current annot. name(1/26): p__

ℹ [2024-06-08 22:18:15] INFO ==> current annot. name(2/26): p__Acidobacteria

ℹ [2024-06-08 22:18:28] INFO ==> current annot. name(3/26): p__Actinobacteria

ℹ [2024-06-08 22:18:41] INFO ==> current annot. name(4/26): p__Armatimonadetes

ℹ [2024-06-08 22:18:53] INFO ==> current annot. name(5/26): p__Bacteroidetes

ℹ [2024-06-08 22:19:06] INFO ==> current annot. name(6/26): p__Chloroflexi

ℹ [2024-06-08 22:19:19] INFO ==> current annot. name(7/26): p__Crenarchaeota

ℹ [2024-06-08 22:19:31] INFO ==> current annot. name(8/26): p__Cyanobacteria

ℹ [2024-06-08 22:19:43] INFO ==> current annot. name(9/26): p__Deinococcus.Thermus

ℹ [2024-06-08 22:19:56] INFO ==> current annot. name(10/26): p__Elusimicrobia

ℹ [2024-06-08 22:20:09] INFO ==> current annot. name(11/26): p__Entotheonellaeota

ℹ [2024-06-08 22:20:20] INFO ==> current annot. name(12/26): p__Euryarchae

In [9]:
# Show markers 
head(dataset.dts.aliyun$abd$mar$correlation)

,var,r,p
,<fct>,<dbl>,<dbl>
p__Crenarchaeota,p__Crenarchaeota,0.2233029,0.001
p__Actinobacteria,p__Actinobacteria,0.1205523,0.001
p__Thaumarchaeota,p__Thaumarchaeota,0.1201680,0.001
p__Gemmatimonadetes,p__Gemmatimonadetes,0.1108208,0.001


In [10]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 3. Calculate alpha diversity indices

We implemented a functiobn of `calc_alpha_div()`, which calculates serveral alpha diversity indices such as observed ASVs and Shannon index. Just run the following codes.

In [11]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_alpha_div(measures = c("observed", "shannon")) %>% suppressMessages()

In [12]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


## 4. Calculate beta diversity distance matrices

We implemented a function of `calc_beta_div()`, which calculates serveral beta diversity distance matrix such as `Bray-Curtis` and `Jarccard` distance matrix. Just run the following codes.

In [13]:
# Calculate alpha diversity indices
dataset.dts.aliyun %<>% calc_beta_div(measures = c("bray", "jaccard")) %>% suppressMessages()

In [14]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


In [15]:
# Check results [Bray-Curtis distance]
dataset.dts.aliyun$div$bet$bray[1:4, 1:4]

,s1,s2,s3,s4
s1,0.0000000,0.4856874,0.5804143,0.4839925
s2,0.4856874,0.0000000,0.4907721,0.4269303
s3,0.5804143,0.4907721,0.0000000,0.4158192
s4,0.4839925,0.4269303,0.4158192,0.0000000


## 5. Calculate microbial community assembly indices

We implemeted a function of `calc_phylo_asmb()`, which calculates the metrics involved in microbial community assembly. Just run the following codes.

In [16]:
# Calculate `alpha.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'alpha.phylo', runs = 9, out.dir = 'test/calc_comm_asmb') %>% suppressMessages()

In [17]:
# Check results 
head(dataset.dts.aliyun$asb$alpha.phylo)

,ntaxa,mntd.obs,mntd.rand.mean,mntd.rand.sd,mntd.obs.rank,mntd.obs.z,mntd.obs.p,runs
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
s1,1025,0.04106696,0.08161756,0.002877478,1,-14.092410,0.1,9
s2,937,0.04935124,0.08603810,0.007130529,1,-5.145040,0.1,9
s3,845,0.04806105,0.09186194,0.007032748,1,-6.228134,0.1,9
s4,1038,0.04639918,0.08223707,0.008971357,1,-3.994701,0.1,9
s5,898,0.04579750,0.08977941,0.007259340,1,-6.058665,0.1,9
s6,973,0.04750401,0.08405180,0.009327828,1,-3.918147,0.1,9


In [18]:
# Calculate `beta.phylo` null model
# runs = 9 just for an example. 999 runs may be better 
dataset.dts.aliyun %<>% calc_phylo_asmb(type = 'beta.phylo', runs = 9, out.dir = 'test/calc_comm_asmb') %>% suppressMessages()

In [19]:
# Check distance matrix 
names(dataset.dts.aliyun$asb$beta.phylo$dis) # distance matrices

[1] "b.mntd" "bc"     "rc"     "b.nti"

In [20]:
# Check the results 
head(dataset.dts.aliyun$asb$beta.phylo$raw$result) # the raw result of `iCAMP::qpen()`

,sample1,sample2,bMNTD,BC,bNTI,RC,process
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,s2,s1,0.01482283,0.4994350,-7.032617,-1,Homogeneous.Selection
2,s3,s1,0.01822827,0.5702448,-4.684807,-1,Homogeneous.Selection
3,s4,s1,0.01598193,0.4775895,-2.093879,-1,Homogeneous.Selection
4,s5,s1,0.01284542,0.4077213,-3.840588,-1,Homogeneous.Selection
5,s6,s1,0.01567921,0.4738230,-5.191145,-1,Homogeneous.Selection
6,s7,s1,0.01544265,0.4815443,-6.252689,-1,Homogeneous.Selection


In [21]:
# Show dataset 
dataset.dts.aliyun %>% show_dataset()

── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$abd$raw: 7 abundance tables (Kingdom, Phylum, Class, Order, Family, Genus, Species)

✔ object$abd$mar: 4 eco-markers at Phylum level (abundance, correlation)

✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)

✔ object$asb$beta.phylo$raw$result: 7 beta phylogenetic assembling index/indices (sample1, sample2, bMNTD, ...)

✔ object$asb$beta.phylo$dis: 4 beta phylogenetic distance matrix/matrices (b.mntd, bc, rc, b.nti)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`
